# Agentics Mini Tutorial

Agentics provides the implementation of AG, a powerful datatype that connects LLMs to pydantic objects and enable logical transuction. 


In [ ]:
! uv pip install agentics-py

## Initialize AGs


In [2]:
from agentics import AG
my_first_agentics = AG()
my_first_agentics.pretty_print()

2025-09-27 12:59:27.591 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


'Atype : None\n'

## Use Agentics as Lists

In [3]:
from agentics import AG
my_first_agentics = AG()

print("The agentics is empty :", len(my_first_agentics))

 ## Add elements to the list
my_first_agentics.append("Alfio")
## internally, agentics stores the elements in the attribute states
my_first_agentics.states += ["Naweed" , "Junkyuu"] 

print("The agentics now has more instances :",len(my_first_agentics))

print("Iterating over agentics:") 
for state in  my_first_agentics:
    print(state)

print("Be careful, the AG itself is not a list :" , my_first_agentics) 

2025-09-27 12:59:27.600 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


The agentics is empty : 0
The agentics now has more instances : 3
Iterating over agentics:
Alfio
Naweed
Junkyuu
Be careful, the AG itself is not a list : atype=None crew_prompt_params={'role': 'Task Executor', 'goal': 'You execute tasks', 'backstory': 'You are always faithful and provide only fact based answers.', 'expected_output': 'Described by Pydantic Type'} instructions='Generate an object of the specified type from the following input.' llm=<crewai.llm.LLM object at 0x310dc2870> max_iter=3 prompt_template=None reasoning=None skip_intentional_definition=False states=['Alfio', 'Naweed', 'Junkyuu'] tools=None transduce_fields=None transduction_logs_path=None transduction_timeout=None verbose_transduction=True verbose_agent=False


## Atypes
Pydantic models can be used to define typed AGs, i.e. whose states are instances of those types. 
To do so let's create a new type and a new AG whose atype is assigned to that

In [4]:
from pydantic import BaseModel
from typing import Optional


class Movie(BaseModel):
    title: Optional[str] = None
    genre: Optional[str] = None
    description: Optional[str] = None


movies = AG(atype=Movie)
movies.append(Movie(title="La dolce vita"))
print(movies.pretty_print())


2025-09-27 12:59:27.612 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class '__main__.Movie'>
title: La dolce vita
genre: null
description: null




## AGs can be imported and exported from json(l) and csv

In [5]:
# Create a new AG object from the provided csv file and a target type
movies = AG.from_csv("data/movies.csv",atype=Movie)

# Note that the type is automatically induced and all arguments are set to srt,
# only matching attributed in columns/atype are imported
print(movies.pretty_print())

2025-09-27 12:59:27.624 | DEBUG    | agentics.core.agentics:from_csv:329 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-27 12:59:27.625 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class '__main__.Movie'>
title: null
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

title: null
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

title: null
genre: Drama, Crime
description: In the continuing saga of the Corleone crime family, a young Vito Corleone
  grows up in Sicily and in 1910s New Yo

## Logical transduction


Once an AG is initilized with a type, Agentics enables us to transduce almost every python object into that. If a list of objects is provided, they will be asyncronously transduced.

### Transduction from Strings
The most basic use of agentics is as an async scaleout framework for llm calls. 

In [6]:
print(await (AG() << "how are you?"))



# from typing import List
# answer= await (AG() << "What are the benefits of using Agentic AI for data workflows?")

2025-09-27 12:59:27.646 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


I'm doing great, thanks for asking! How can I help you today?


In [7]:
class Answer(BaseModel):
    answer: Optional[str] = None
    justification: Optional[str] = None
    confidence: Optional[float] = None

answer= await (AG(atype=Answer) << "What are the benefits of using Agentic AI for data workflows?")

## Note that agentics are always list, 
## so the object returned is a list and the first element is selected. 
print(answer[0])

2025-09-27 12:59:29.336 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


answer='Agentic AI brings several key benefits to data workflows:\n1. **Automation of repetitive tasks** – It can autonomously extract, clean, transform, and load data without constant human oversight.\n2. **Adaptive decision‑making** – By continuously learning from new data, the agent can adjust pipelines on‑the‑fly to handle schema changes, anomalies, or evolving business rules.\n3. **Self‑optimization** – Agentic systems can monitor performance metrics (e.g., latency, error rates) and re‑configure steps to improve efficiency and cost.\n4. **Scalability** – Agents can spin up additional compute resources or parallelize operations as data volume grows, ensuring consistent throughput.\n5. **Improved data quality** – Through built‑in validation, anomaly detection, and corrective actions, the AI maintains higher accuracy and completeness of datasets.\n6. **Faster time‑to‑insight** – By reducing manual bottlenecks and accelerating pipeline execution, stakeholders receive actionable analyt

### Transduction from lists

If the source is a list of strings, all transduction will be executed asyncronously and will return a list of states. 

In [8]:
answers= await (AG(atype=Answer) << ["What are the benefits of using Agentic AI for data workflows?",
                                   "Will AI improve working conditions for middle class?"])

## Note that agentics are always list, 
## so the object returned is a list and the first element is selected. 
print(answers.pretty_print())

2025-09-27 12:59:32.555 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.Answer'>
answer: "Agentic AI brings several key benefits to data workflows:\n- **Automation\
  \ of repetitive tasks**: It can autonomously handle data extraction, cleaning, transformation,\
  \ and loading without constant human oversight.\n- **Adaptive decision\u2011making**:\
  \ By perceiving the state of the workflow and acting on it, the agent can choose\
  \ the most appropriate tools or methods for each step, adjusting to changing data\
  \ conditions.\n- **Self\u2011optimization**: Agentic systems can monitor performance\
  \ metrics and iteratively refine their own processes, leading to faster and more\
  \ efficient pipelines.\n- **Scalability**: Agents can be deployed in parallel across\
  \ multiple datasets or environments, scaling out as data volume grows.\n- **Improved\
  \ data quality**: Continuous monitoring and corrective actions help detect anomalies,\
  \ missing values, or inconsistencies early, ensuring higher-quality outputs.\n-\
  \ **

### Transduction from python objects
In Agentics, transduction is not limited to Pydantic types. Any Python object can serve as a source for transduction. This ensures that the object can be rendered into a textual form interpretable by the LLM, while the target AG enforces schema validation on the structured output.

Note: Agentics internally call the __str__() method in the object’s class. If __str__() is not defined, it falls back to __repr__(). Default methods for many objects might be meaningless. 

In [9]:
from pydantic import Field, BaseModel
from typing import Optional
from agentics import AG

class ObjectDescription(BaseModel):
    "This is a description of the input object. It could be the str "
    type: Optional[str] = Field("str", description="The python datatype or object that is being printed, e.g. str, dict, list, list[str]")
    description: Optional[str] = None

import numpy as np
arr = np.random.rand(3, 3)


predictions = await (AG(atype=ObjectDescription) << [arr, 
                                                     0.334, 
                                                    answers.states[0]])
print(predictions.pretty_print())

2025-09-27 12:59:35.882 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.ObjectDescription'>
type: numpy.ndarray
description: 3x3 array of float values

type: float
description: A floating point number representing the value 0.334

type: Agentic AI Benefits
description: "Agentic AI brings several key benefits to data workflows:\n- **Automation\
  \ of repetitive tasks**: It can autonomously handle data extraction, cleaning, transformation,\
  \ and loading without constant human oversight.\n- **Adaptive decision\u2011making**:\
  \ By perceiving the state of the workflow and acting on it, the agent can choose\
  \ the most appropriate tools or methods for each step, adjusting to changing data\
  \ conditions.\n- **Self\u2011optimization**: Agentic systems can monitor performance\
  \ metrics and iteratively refine their own processes, leading to faster and more\
  \ efficient pipelines.\n- **Scalability**: Agents can be deployed in parallel across\
  \ multiple datasets or environments, scaling out as data volume grows.\n- **Improve

### Transduction between AGs

Transduction can be better controlled and instructed when executed between two AGs. In this case, the list of states of the source AG will be rendered by the prompt function and will be sent to the target AG. 

In [10]:
from typing import Literal, Optional
from pydantic import BaseModel
from agentics import AG

class ProductReview(BaseModel):
    reviewer: Optional[str] = None
    text: Optional[str] = None
    stars: Optional[int] = None
    
class SentimentSummary(BaseModel):
    customer_sentiment: Optional[Literal["positive", "neutral", "negative"]] = None
    reason: Optional[str] = None

product_reviews= [ProductReview(reviewer = "Alice", text = "Excellent product quality and fast delivery!", 
                  stars = 5),
                ProductReview(reviewer = "Bob", text = "It’s okay, but the package was damaged", stars =  3),
                ProductReview(reviewer = "Carol", text =  "Terrible experience, broken after one use!", stars =  1)]

source = AG(atype=ProductReview, states = product_reviews)
target = AG(atype=SentimentSummary)

transductions = await (target<< source)
print(transductions.pretty_print())

2025-09-27 12:59:41.285 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-27 12:59:41.285 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.SentimentSummary'>
customer_sentiment: positive
reason: The reviewer gave a 5-star rating and praised the product quality and fast
  delivery, indicating strong positive sentiment.

customer_sentiment: neutral
reason: Package was damaged

customer_sentiment: negative
reason: Product broke after one use




### Self transduction

Self transduction can be done across different subsets of fields of the same atype, providing a convenient way to operate on dataframes and state graphs. 

In [11]:
movies = AG.from_csv("data/movies.csv",atype=Movie)
self_transductions = await movies.self_transduction(["title","description"],["genre"])
print(self_transductions.pretty_print())

2025-09-27 12:59:42.657 | DEBUG    | agentics.core.agentics:from_csv:329 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-27 12:59:42.658 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.Movie'>
title: null
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

title: null
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

title: null
genre: Drama, Crime
description: In the continuing saga of the Corleone crime family, a young Vito Corleone
  grows up in Sicily and in 1910s New Yo

## Tool usage
Agentics is fully compatible with the MCP ecosystem and can use them as tools. 

In [12]:
from mcp import StdioServerParameters  # For Stdio Server
import os
from crewai_tools import MCPServerAdapter

server_params_fetch = StdioServerParameters(
    command="uvx",
    args=["mcp-server-fetch"],
    env={"UV_PYTHON": "3.12", **os.environ})

with (MCPServerAdapter(server_params_fetch) as server_fetch):
    print(f"Available tools from Stdio MCP server: {[tool.name for tool in server_fetch]}")

class Website_Description(BaseModel):
    url:Optional[str] = None
    content_summary:Optional[str] = None

description = await (AG(atype=Website_Description,
                        tools=server_fetch) \
                      << "Provide a short summary of the content from this website https://www.vldb.org/2026/call-for-research-track.html")

print(description.pretty_print())

2025-09-27 12:59:59.639 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Available tools from Stdio MCP server: ['fetch']
Atype : <class '__main__.Website_Description'>
url: https://www.vldb.org/2026/call-for-research-track.html
content_summary: "The VLDB 2026 Call for Research Track invites submissions of original,\
  \ high\u2011quality research papers covering all aspects of data management and\
  \ database systems. Submissions must be full papers (up to 12 pages, excluding references)\
  \ and are subject to a double\u2011blind review process. Important dates include\
  \ a submission deadline of March 15, 2026, with notifications by May 1, 2026, and\
  \ final camera\u2011ready versions due June 1, 2026. Accepted papers will be presented\
  \ at the VLDB 2026 conference in Tokyo, Japan. The track welcomes contributions\
  \ on topics such as data storage, query processing, transaction management, data\
  \ integration, machine learning for data management, and emerging data\u2011centric\
  \ technologies. Authors should follow the provided LaTeX/Word 

## a Map Reduce Framework
In addition to the defauls async behaviour of logical transduction, Agentics extends the same behavious to the execution of any function which operate on its states. This enables the definition of scalable and composable workflows. 

### AMap
The method AG.amap(func) executed the provided fuction throug all states. 
func is an asyc function that takes objects of type atype and return objects of the same type
